In [ ]:
from statistics import median
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Leer los datos del archivo JSON
data_raw = pd.read_json('./Ventas_registro.json')

In [ ]:
# Desanidamos los productos para usar los valores anidados (nombre - cantidad - total)
data_flat = pd.json_normalize(data_raw.to_dict(orient='records'), record_path='productos', meta=['tipo_venta', 'fecha_venta', 'pago_total', 'cliente'])

### TEMA 8

#### DEFINICIÓN DE VARIABLES CANTIDAD, TOTAL, PAGO_TOTAL

In [ ]:
sns.boxplot(data=data_flat, x='cantidad', legend='auto')
plt.title('Diagrama de Caja - cantidad de productos')
plt.xlabel('Total de cantidad por Producto')
plt.show()

In [ ]:
sns.boxplot(data=data_flat, x='total', legend='auto')
plt.title('Diagrama de Caja - Total de productos')
plt.xlabel('Total de Ventas por Producto')
plt.show()

In [ ]:
sns.boxplot(data=data_flat, x='pago_total', legend='auto')
plt.title('Diagrama de Caja - Pago total')
plt.xlabel('Pago total de ventas')
plt.show()

#### Z-SCORE

In [ ]:
from scipy.stats import zscore

# Calcular el Z-score para `cantidad` y `total`
data_flat['z_cantidad'] = zscore(data_flat['cantidad'])
data_flat['z_total'] = zscore(data_flat['total'])

# Filtrar los outliers (valores cuyo Z-score > 3 o <-3)
data_flat_clean = data_flat[(data_flat['z_cantidad'].abs() <= 3) & (data_flat['z_total'].abs() <= 3)]

# Eliminar las columnas z_cantidad y z_total para mantener la estructura original
data_flat_clean = data_flat_clean.drop(['z_cantidad', 'z_total'], axis=1)

# Ver los primeros registros de la tabla limpia
print(data_flat_clean.shape)

In [ ]:
from scipy.stats import zscore

# Calcular el Z-score para la columna `pago_total`
data_raw['z_pago_total'] = zscore(data_raw['pago_total'])

# Filtrar los outliers (valores cuyo Z-score > 3 o <-3)
data_raw_clean = data_raw[data_raw['z_pago_total'].abs() <= 3]

# Eliminar la columna z_pago_total para mantener la estructura original
data_raw_clean = data_raw_clean.drop(['z_pago_total'], axis=1)

# Ver los primeros registros de la tabla limpia
print(data_raw_clean.shape)

In [ ]:
# Boxplot de `pago_total` antes y después de limpiar outliers
plt.figure(figsize=(12, 6))

# Boxplot para `pago_total` en data_raw (antes de limpiar)
sns.boxplot(x='status', y='pago_total', data=pd.DataFrame({
    'pago_total': data_raw['pago_total'],
    'status': ['Antes'] * len(data_raw)
}), color='lightblue', width=0.4)

# Boxplot para `pago_total` en data_raw_clean (después de limpiar)
sns.boxplot(x='status', y='pago_total', data=pd.DataFrame({
    'pago_total': data_raw_clean['pago_total'],
    'status': ['Después'] * len(data_raw_clean)
}), color='lightgreen', width=0.4)

plt.title('Comparativa de Boxplot de pago_total: Antes vs Después de limpiar outliers')
plt.ylabel('Pago Total')
plt.show()

In [ ]:
# Boxplot para `cantidad` en data_flat (antes y después de limpiar outliers)
plt.figure(figsize=(12, 6))

# Boxplot para `cantidad` en data_flat (antes de limpiar)
sns.boxplot(x='status', y='cantidad', data=pd.DataFrame({
    'cantidad': data_flat['cantidad'],
    'status': ['Antes'] * len(data_flat)
}), color='lightblue', width=0.4)

# Boxplot para `cantidad` en data_flat_clean (después de limpiar)
sns.boxplot(x='status', y='cantidad', data=pd.DataFrame({
    'cantidad': data_flat_clean['cantidad'],
    'status': ['Después'] * len(data_flat_clean)
}), color='lightgreen', width=0.4)

plt.title('Comparativa de Boxplot de cantidad: Antes vs Después de limpiar outliers')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
# Boxplot para `total` en data_flat (antes y después de limpiar outliers)
plt.figure(figsize=(12, 6))

# Boxplot para `total` en data_flat (antes de limpiar)
sns.boxplot(x='status', y='total', data=pd.DataFrame({
    'total': data_flat['total'],
    'status': ['Antes'] * len(data_flat)
}), color='lightblue', width=0.4)

# Boxplot para `total` en data_flat_clean (después de limpiar)
sns.boxplot(x='status', y='total', data=pd.DataFrame({
    'total': data_flat_clean['total'],
    'status': ['Después'] * len(data_flat_clean)
}), color='lightgreen', width=0.4)

plt.title('Comparativa de Boxplot de total: Antes vs Después de limpiar outliers')
plt.ylabel('Total')
plt.show()

### TEMA 9

#### PREPARACIÓN DE DATOS - CONSTRUCCIÓN

##### GENERACIÓN DE DATOS

1. ATRIBUTOS DERIVADOS

In [ ]:
# Calcular el precio unitario de cada producto
data_flat_clean['precio_unitario'] = data_flat_clean['total'] / data_flat_clean['cantidad']
print(data_flat_clean[['nombre', 'cantidad', 'total', 'precio_unitario']].head())

In [ ]:
# Total de productos vendidos en una venta
data_raw_clean['total_productos'] = data_flat_clean.groupby(['fecha_venta', 'cliente'])['cantidad'].transform('sum')
data_raw_clean.head()

##### TRANSFORMACIÓN DE DATOS

In [ ]:
# Formatear el tipo de dato fecha_venta a datetime64
data_raw_clean['fecha_venta'] = pd.to_datetime(data_raw_clean['fecha_venta'], format='%d/%m/%Y')
data_flat_clean['fecha_venta'] = pd.to_datetime(data_flat_clean['fecha_venta'], format='%d/%m/%Y')
print(data_flat_clean['fecha_venta'].head())

1. AGREGACIÓN

In [ ]:
# Calcular ventas mensuales por producto ¿?
data_flat_clean['mes'] = pd.to_datetime(data_flat_clean['fecha_venta']).dt.to_period('M')
ventas_mensuales = data_flat_clean.groupby(['mes', 'nombre'])['cantidad'].sum().reset_index()
data_flat_clean.head(500)

2. CONSTRUCCIÓN DE ATRIBUTOS

In [86]:
# Creación de un nuevo atributo par tipo_venta
data_raw_clean['tipo_venta_label'] = data_raw_clean['tipo_venta'].map({0: 'Estándar', 1: 'Promoción'})
data_raw_clean.head()

,tipo_venta,fecha_venta,productos,pago_total,cliente,total_productos,tipo_venta_label
0,0,2021-03-11,"[{'nombre': 'Pantalones capri', 'cantidad': 3,...",990,Elias Medina,9.0,Estándar
1,1,2021-03-04,"[{'nombre': 'Suerte oversized blanco', 'cantid...",615,Gabriela Campos,9.0,Promoción
2,0,2021-02-06,"[{'nombre': 'pantalone chino azul', 'cantidad'...",70,Simon Luna,9.0,Estándar
3,1,2021-02-19,"[{'nombre': 'Top cruzado', 'cantidad': 3, 'tot...",215,Jose Torres,9.0,Promoción
4,0,2021-01-27,"[{'nombre': 'Blusa aflorado negro', 'cantidad'...",305,Ramiro Guzman,7.0,Estándar


### TEMA 10